### Set up the Huggingface Pipeline

In [12]:
import transformers
import torch

model_name = 'sc-genrm-scaling/llama_3.1_8b_genrm_ft'

pipeline = transformers.pipeline(
    "text-generation",
    model=model_name,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

### Prepare the input prompt

In [19]:
# load the prompt template
prompt_template_path = 'prompts/llama3.1_8b_instruct/finetuned.txt'

with open(prompt_template_path, "r") as file:
    prompt_template = file.read()

# insert the problem and solution in the template
question = "Let $\\mathbf{a},$ $\\mathbf{b},$ and $\\mathbf{c}$ be three vectors\
  \ such that $\\|\\mathbf{a}\\| = \\|\\mathbf{b}\\| = \\|\\mathbf{c}\\| = 2.$  Also,\
  \ the angle between any two of these vectors is $\\arccos \\frac{5}{8}.$  Find the\
  \ volume of the parallelepiped generated by $\\mathbf{a},$ $\\mathbf{b},$ and $\\\
  mathbf{c}.$"

solution = "The volume of the parallelepiped generated by $\\mathbf{a},$\
  \ $\\mathbf{b},$ and $\\mathbf{c}$ is equal to $\\left|\\mathbf{a}\\cdot(\\mathbf{b}\\\
  times\\mathbf{c})\\right|.$  Since the angle between any two of the vectors is $\\\
  arccos\\frac{5}{8},$ we have\n\n\\begin{align*}\n\\left|\\mathbf{a}\\cdot(\\mathbf{b}\\\
  times\\mathbf{c})\\right|&=|\\mathbf{a}|\\cdot|\\mathbf{b}\\times\\mathbf{c}|\\\
  cos\\theta\\\\\n&=|\\mathbf{a}|\\cdot\\left|\\mathbf{b}\\right|\\cdot\\left|\\mathbf{c}\\\
  right|\\sin\\phi\\cos\\theta\\\\\n&=(2)(2)(2)\\cdot\\frac{21}{64}\\cdot\\frac{5}{8}\\\
  \\\n&=\\boxed{\\frac{105}{32}}.\n\\end{align*}\nFinal Answer: The final answer is\
  \ $\\frac{105}{32}$. I hope it is correct."

prompt = prompt_template.format(question, solution)

messages = [
    {"role": "user", "content": prompt},
]

In [21]:
print(messages)

[{'role': 'user', 'content': 'Problem:\nLet $\\mathbf{a},$ $\\mathbf{b},$ and $\\mathbf{c}$ be three vectors  \\ such that $\\|\\mathbf{a}\\| = \\|\\mathbf{b}\\| = \\|\\mathbf{c}\\| = 2.$  Also,  \\ the angle between any two of these vectors is $\\arccos \\frac{5}{8}.$  Find the  \\ volume of the parallelepiped generated by $\\mathbf{a},$ $\\mathbf{b},$ and $\\  mathbf{c}.$\n\nSolution:\nThe volume of the parallelepiped generated by $\\mathbf{a},$  \\ $\\mathbf{b},$ and $\\mathbf{c}$ is equal to $\\left|\\mathbf{a}\\cdot(\\mathbf{b}\\  times\\mathbf{c})\\right|.$  Since the angle between any two of the vectors is $\\  arccos\\frac{5}{8},$ we have\n\n\\begin{align*}\n\\left|\\mathbf{a}\\cdot(\\mathbf{b}\\  times\\mathbf{c})\\right|&=|\\mathbf{a}|\\cdot|\\mathbf{b}\\times\\mathbf{c}|\\  cos\\theta\\\\\n&=|\\mathbf{a}|\\cdot\\left|\\mathbf{b}\\right|\\cdot\\left|\\mathbf{c}\\  right|\\sin\\phi\\cos\\theta\\\\\n&=(2)(2)(2)\\cdot\\frac{21}{64}\\cdot\\frac{5}{8}\\  \\\n&=\\boxed{\\frac{105}{

### Sample Output

In [23]:
outputs = pipeline(
    messages,
    max_new_tokens=2048,
    temperature=0.7,
    top_p=0.95,
    top_k=40
)
print(outputs[0]["generated_text"][-1]['content'])

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


### Step-by-Step Verification

1. **Problem Understanding**:
   - The problem asks for the volume of the parallelepiped generated by three vectors $\mathbf{a},$ $\mathbf{b},$ and $\mathbf{c},$ where the vectors have a magnitude of $2$ and the angle between any two vectors is $\arccos\frac{5}{8}.$
   - The volume is given by the absolute value of the triple scalar product $\left|\mathbf{a}\cdot(\mathbf{b}\times\mathbf{c})\right|.$

2. **Equation Setup**:
   - The solution starts with the correct expression for the volume: $\left|\mathbf{a}\cdot(\mathbf{b}\times\mathbf{c})\right|.$
   - The next step involves expanding this expression using the properties of the dot product and cross product:
     \[
     \left|\mathbf{a}\cdot(\mathbf{b}\times\mathbf{c})\right| = |\mathbf{a}| \cdot |\mathbf{b} \times \mathbf{c}| \cdot \cos\theta.
     \]

   - This is correct, as the magnitude of the cross product $\mathbf{b}\times\mathbf{c}$ gives the area of the parallelogram formed by $\mathbf{b}$ and